In [5]:
import torch
import torch.nn as nn
import json
from datetime import datetime
from tqdm import tqdm

import os
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
import more_itertools as mit
import time
import numpy as np

from spot import SPOT
from mtad_gat import MTAD_GAT
from trainer import Trainer
from predictor import Predictor
from utils import *
#from torch.utils.tensorboard import SummaryWriter
#da cambiare tensorboard con WandB

In [6]:
# da definire tutti manualmente, poi vedere valori di default in args

#dataset = args.dataset
window_size = 100
spec_res = False
normalize = False
n_epochs = 50
batch_size = 512
init_lr = 1e-3
val_split = 0.1
shuffle_dataset = False
use_cuda = True
print_every = 1
log_tensorboard = True
#group_index = args.group[0]True
#index = args.group[2:]
#args_summary = str(args.__dict__)
#print(args_summary)

In [3]:
df = get_data()

timestamp = df["timestamp"] #salvo il timestamp per dopo
#df.set_index("timestamp", inplace=True)

to_drop = [
    "sin",
    "cos",
    "2m",
    "soil",
    "rain",
    "timestamp"
]

for col in df:
    if all(key in col for key in to_drop):
        df.drop(columns=col, inplace=True)
    

#df.drop(columns="timestamp, ", inplace=True)
train_data = df.to_numpy()
x_dim = len(df.columns)# numero di colonne
train_data = train_data.reshape((-1, x_dim))#[train_start:train_end, :]
    
x_train, x_test = train_test_split(train_data, train_size=0.8)

# converte il dataframe in un tensore
x_train = torch.from_numpy(x_train).float()

x_test = torch.from_numpy(x_test).float()
# Il secondo argomento del tensore sono il numero di colonne del dataframe
n_features = x_train.shape[1]
out_dim = n_features
target_dims = None # if none models all features
# si crea la cartella di output
log_dir = './logs'
save_path = "./fv_model"


NameError: name 'get_data' is not defined

In [11]:
# Inizializza il modello
model = MTAD_GAT(
    n_features,
    window_size,
    out_dim,
    kernel_size=7,
    use_gatv2=True,
    feat_gat_embed_dim=None,
    time_gat_embed_dim=None,
    gru_n_layers=1,
    gru_hid_dim=150,
    forecast_n_layers=3,
    forecast_hid_dim=150,
    recon_n_layers=1,
    recon_hid_dim=150,
    dropout=0.3,
    alpha=0.2
)

# inizializza optimizer e robe varie
optimizer = torch.optim.Adam(model.parameters(), lr=init_lr)
forecast_criterion = nn.MSELoss()
recon_criterion = nn.MSELoss()

# altre robe di pytorch
trainer = Trainer(
    model,
    optimizer,
    window_size,
    n_features,
    target_dims,
    n_epochs,
    batch_size,
    init_lr,
    forecast_criterion,
    recon_criterion,
    use_cuda,
    save_path,
    log_dir,
    print_every,
    log_tensorboard=False
)

In [12]:
trainer.load(f"{save_path}/model.pt")
prediction_args = {
    "target_dims": target_dims,
    'scale_scores': False,
    "level": 0.95,
    "q": 0.005,
    'dynamic_pot': False,
    "use_mov_av": False,
    "gamma": 1,
    "reg_level": 1,
    "save_path": "./results/",
}
best_model = trainer.model
predictor = Predictor(
    best_model,
    window_size,
    n_features,
    prediction_args,
)
y_test=None
label = y_test[window_size:] if y_test is not None else None
predictor.predict_anomalies(x_train, x_test, label)


Predicting and calculating anomaly scores..


100%|██████████| 58/58 [00:00<00:00, 59.18it/s]


Predicting and calculating anomaly scores..


100%|██████████| 15/15 [00:00<00:00, 115.82it/s]
/home/jovyan/anomaly-detection-no-gpu-volume-30/deep/spot.py:305: RuntimeWarning: divide by zero encountered in log
  return 1 + np.log(s).mean()
/home/jovyan/anomaly-detection-no-gpu-volume-30/deep/spot.py:308: RuntimeWarning: divide by zero encountered in true_divide
  return np.mean(1 / s)
/home/jovyan/anomaly-detection-no-gpu-volume-30/deep/spot.py:321: RuntimeWarning: divide by zero encountered in true_divide
  jac_vs = (1 / t) * (-vs + np.mean(1 / s ** 2))
/home/jovyan/anomaly-detection-no-gpu-volume-30/deep/spot.py:321: RuntimeWarning: invalid value encountered in float_scalars
  jac_vs = (1 / t) * (-vs + np.mean(1 / s ** 2))


Running POT with q=0.005, level=0.95..
Initial threshold : 0.4997896
Number of peaks : 729
Grimshaw maximum log-likelihood estimation ... [done]
	γ = -0.32017773389816284
	σ = 0.05192852913196731
	L = 1660.7091658930503
Extreme quantile (probability = 0.005): 0.5843481942861153


100%|██████████| 3641/3641 [00:00<00:00, 4375776.75it/s]

0
3641
Results using epsilon method:
 {'threshold': 0.6116647720336914, 'reg_level': 1}
Results using peak-over-threshold method:
 {'threshold': 0.5843481942861154}
Results using best f1 score search:
 {}
Saving output to ./results//<train/test>_output.pkl
-- Done.
